# Preprocessing Neurophysiology Data: Create json-files to Run
[ReTune Dyskinesia Project]


This notebook contains a step-by-step overview of the preprocessing workflow for ECoG- and LFP-data within the ReTune-Project work package B04. This step-wise structure is provided to understand, visualize, and adjust the single steps. Besides this notebook, another script provides execution of the preprocessing steps at once.


<b> Data is required to converted into the BIDS-standard. </b>


### 0. Loading packages and functions, defining paths



In [198]:
# Importing Python and external packages
import os
import json


In [5]:
# define local storage directories
projectpath = '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'
codepath = os.path.join(projectpath, 'code')
rawdatapath = '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata'
json_dir = os.path.join(
    projectpath, 'data', 'preprocess', 'preprocess_jsons'
)
# change working directory to project-code folder
os.chdir(codepath)
os.getcwd()

'/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code'

In [3]:
import lfpecog_preproc.preproc_json_creator as json_creator

# import lfpecog_preproc.preproc_data_management as dataMng
# import lfpecog_preproc.preproc_reref as reref
# import lfpecog_preproc.preproc_artefacts as artefacts
# import lfpecog_preproc.preproc_filters as fltrs
# import lfpecog_preproc.preproc_resample as resample

### 1. Define Settings of Preprocessing and JSON-file

In [142]:
def define_settings(
    version: str, create: bool, json_dir: str,
    ecog_sett_list: list = None, lfp_sett_list: list = None,
    plot_figures: bool = False, lfp_reref: str = None
):
    '''
    Function...

    Arguments:
        - version: name of settings-version
        - create: To create new or not (used existing)
        - json_dir: path to json-folder
        - ecog_sett_list/ lfp_sett_list: Lists with new settings
        to make for ecog and lfp preprocessing:
        [win_len, artfct_sd_tresh,
        (bandpass_f_low, bandpass_f_low), transBW, notchW,
        Fs_origin, Fs_resample, settings_version]

    '''
    f = f'settings_{version}.json'
    if create:
        # check for already existing
        if f in os.listdir(json_dir):
                print(f'\nSettings-JSON {version} does already exist,'
                ' define new name for current version')
                return
        for newlist in [ecog_sett_list, lfp_sett_list]:
            if newlist is None:  # check if new list is inserted
                print('Enter list(s) with new settings')
                return
            if (float or str) in [type(s) for s in newlist]:  # only int
                print('List with new settings can only contain integers')
                return
        if lfp_reref not in ['segments', 'levels']:
            print('LFP-reref method has to be levels / segments')
            return

        # if all requirements are met: create new json
        dict_settings = {  # dict to write into json
            'lfp': lfp_sett_list + [version],
            'ecog': ecog_sett_list + [version],
            'plot_figs': plot_figures,
            'lfp_reref': lfp_reref,
        }
        with open(os.path.join(json_dir, f), 'w') as jsonfile:
            json.dump(dict_settings, jsonfile, indent=4)
        return os.path.join(json_dir, f)

    if not create:
        # Define existing version to use
        if f not in os.listdir(json_dir):
            print(f'Setting-json {version} does not exist.\n'
                  'Insert existing settings-version!!')
            return
        else:
            print(f'Settings {version} exist, please proceed :)')
            return os.path.join(json_dir, f)


In [200]:
lfp_settings = [1, 4, (1, 120), 10, 2, 4000, 800,]
ecog_settings = [1, 4, (1, 120), 10, 2, 4000, 800,]

json_settings = define_settings(
    'v0.5_Feb22', create=True, json_dir=json_dir,
    ecog_sett_list=ecog_settings, lfp_sett_list=lfp_settings,
    plot_figures=True, lfp_reref='segments'
)

### 2. Select Recordings to Preprocess




In [183]:
# define sub and ses and check available runs
sub = '009'
ses = 'MedOn02'
ses_path = os.path.join(
    rawdatapath,
    f'sub-{sub}',
    f'ses-EcogLfp{ses}/ieeg'
)
fs = [f for f in os.listdir(ses_path) if f[-4:] == '.eeg']
print(f'Available runs for pt {sub} in session {ses}:')
for f in fs: print('\n', f.split('_')[2:5])


Available runs for pt 009 in session MedOn02:

 ['task-SelfpacedHandFlipL', 'acq-StimOffDopa18', 'run-1']

 ['task-Rest', 'acq-StimOffDopa00', 'run-1']

 ['task-SelfpacedHandFlipR', 'acq-StimOffDopa40', 'run-1']

 ['task-Rest', 'acq-StimOffDopa65', 'run-1']

 ['task-SelfpacedHandFlipR', 'acq-StimOffDopa10', 'run-1']

 ['task-SelfpacedHandFlipL', 'acq-StimOffDopa50', 'run-1']

 ['task-Free', 'acq-StimOffDopa55', 'run-1']

 ['task-Free', 'acq-StimOffDopa25', 'run-1']

 ['task-SelfpacedHandFlipR', 'acq-StimOffDopa70', 'run-1']

 ['task-Rest', 'acq-StimOffDopa30', 'run-1']


In [184]:
print('Exclusion from Preprocessing:')
sel_excl = []
to_excl = []
for f_ex in sel_excl:
    to_excl.append(fs[f_ex])
    print(f'{fs[2]} added to exclude from preprocessing')

Exclusion from Preprocessing:


In [185]:
# for f_ex in to_excl: fs.remove(f_ex)
print(f'Files ({len(fs)}) left after exclusion: {fs}')

Files (10) left after exclusion: ['sub-009_ses-EcogLfpMedOn02_task-SelfpacedHandFlipL_acq-StimOffDopa18_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-Rest_acq-StimOffDopa00_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-SelfpacedHandFlipR_acq-StimOffDopa40_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-Rest_acq-StimOffDopa65_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-SelfpacedHandFlipR_acq-StimOffDopa10_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-SelfpacedHandFlipL_acq-StimOffDopa50_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-Free_acq-StimOffDopa55_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-Free_acq-StimOffDopa25_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-SelfpacedHandFlipR_acq-StimOffDopa70_run-1_ieeg.eeg', 'sub-009_ses-EcogLfpMedOn02_task-Rest_acq-StimOffDopa30_run-1_ieeg.eeg']


In [196]:
list_dicts = []
for f in fs:
    list_dicts.append({  # dict to write into json
        'sub': f.split('-')[1][:3],
        'ses': f.split('-')[2][:-5],
        'task': f.split('-')[3][:-4],
        'acq': f.split('-')[4][:-4],
        'run': f.split('-')[5][0],
        'raw_path': rawdatapath,
        'project_path': projectpath,
    })

print('Check dictionaries of included runs', list_dicts)

Check dictionaries of included runs [{'sub': '009', 'ses': 'EcogLfpMedOn02', 'task': 'SelfpacedHandFlipL', 'acq': 'StimOffDopa18', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}, {'sub': '009', 'ses': 'EcogLfpMedOn02', 'task': 'Rest', 'acq': 'StimOffDopa00', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}, {'sub': '009', 'ses': 'EcogLfpMedOn02', 'task': 'SelfpacedHandFlipR', 'acq': 'StimOffDopa40', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}, {'sub': '009', 'ses': 'EcogLfpMedOn02', 'task': 'Rest', 

In [197]:
# Write JSON-file with run-dict's
fname = 'runinfos_08FEB22.json'
f = os.path.join(json_dir, fname)
with open(f, 'w') as jsonfile:
    json.dump(list_dicts, jsonfile, indent=4)